In [0]:
import urllib.request
import shutil
import os
from datetime import datetime, date, timezone
from dateutil.relativedelta import relativedelta

In [0]:
# 3 mois de moins
three_months_ago = date.today() - relativedelta(months=3)
fomatted_month = three_months_ago.strftime("%Y-%m")

# path for the month
dir_path = f"/Volumes/nyctaxi/00_landing/data_sources/nyctaxi_yellow/{fomatted_month}"

# path for the file
local_path = f"{dir_path}/yellow_tripdata_{fomatted_month}.parquet"

# test=dbutils.fs(local_path)

try:
# si le fichier existe déja, on ne doit pas continuer
    dbutils.fs.ls(local_path)
    dbutils.jobs.taskValues.set(key="continue_downstream", value="no")
    print("file already exists, aborting downstream tasks")
# si le fichier n'existe pas, on peut continuer
except:
    try:
    # construis l"URL
        url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{fomatted_month}.parquet"
    # ouvre une connection et stream le fichier
        response = urllib.request.urlopen(url)
    # crée le dossier
        os.makedirs(dir_path, exist_ok=True) 
    # save the stremaed content to the local file in binary mode
        with open(local_path, "wb") as out_file:
            shutil.copyfileobj(response, out_file)
    # set continue downstream to yes
        dbutils.jobs.taskValues.set(key="continue_downstream", value="yes")
        print("file downloaded, continuing downstream tasks")
    except Exception as e :
    # set continue downstream to no if we have an error
        print(f"file not found, aborting downstream tasks. error is: {str(e)}")
        dbutils.jobs.taskValues.set(key="continue_downstream", value="no")
